In [66]:
import pandas as pd
import json
import requests
import datetime


In [62]:
## stock info dataset
api_key = '112fd4c9fef462d4ff99217055be9015'

url = f"http://api.marketstack.com/v1/eod?access_key={api_key}&symbols=VOO&date_from=2015-01-01&date_to=2025-12-12&limit=1000"


response = requests.get(url)
if response.status_code != 200:
    print(f"Error: Received status code {response.status_code}, error: {response.json().get('error', 'Unknown error')}")
else:

    print("Request successful.")
    


Request successful.


### bronze level data

In [63]:
raw = response.json()
#dfRaw = pd.json_normalize(raw['data'])
dfRaw = pd.DataFrame(raw['data'])
dfRaw.to_csv('stock_info.csv', index=False)
raw['pagination']

{'limit': 1000, 'offset': 0, 'count': 251, 'total': 251}

In [64]:
dfRaw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 251 entries, 0 to 250
Data columns (total 15 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   open          251 non-null    float64
 1   high          251 non-null    float64
 2   low           251 non-null    float64
 3   close         251 non-null    float64
 4   volume        251 non-null    float64
 5   adj_high      250 non-null    float64
 6   adj_low       250 non-null    float64
 7   adj_close     251 non-null    float64
 8   adj_open      250 non-null    float64
 9   adj_volume    250 non-null    float64
 10  split_factor  251 non-null    float64
 11  dividend      251 non-null    float64
 12  symbol        251 non-null    object 
 13  exchange      251 non-null    object 
 14  date          251 non-null    object 
dtypes: float64(12), object(3)
memory usage: 29.5+ KB


In [65]:
dfRaw.head()

,open,high,low,close,volume,adj_high,adj_low,adj_close,adj_open,adj_volume,split_factor,dividend,symbol,exchange,date
0,613.07,616.205,605.86,611.43,6370303.0,NaN,NaN,611.43,NaN,NaN,1.0,0.0,VOO,ARCX,2025-10-15T00:00:00+0000
1,604.20,612.140,600.51,608.75,7483600.0,612.1400,600.510,608.75,604.20,7493479.0,1.0,0.0,VOO,ARCX,2025-10-14T00:00:00+0000
2,607.37,611.030,606.58,609.61,6955600.0,611.0272,606.575,609.61,607.37,6968434.0,1.0,0.0,VOO,ARCX,2025-10-13T00:00:00+0000
3,617.99,619.600,600.23,600.51,13600000.0,619.6000,600.230,600.51,617.99,13664157.0,1.0,0.0,VOO,ARCX,2025-10-10T00:00:00+0000
4,619.23,619.590,615.28,617.10,5891900.0,619.5900,615.280,617.10,619.23,5898496.0,1.0,0.0,VOO,ARCX,2025-10-09T00:00:00+0000


In [74]:
dfRaw.to_csv('bronze_stock_info.csv', index=False)

#### Silver Level
    column transformations and imputation

In [ ]:
### date column transformations for interpretability
dfRaw['month'] = pd.to_datetime(dfRaw['date']).dt.month
dfRaw['year'] = pd.to_datetime(dfRaw['date']).dt.year
dfRaw['day'] = pd.to_datetime(dfRaw['date']).dt.day
dfRaw['day_of_week'] = pd.to_datetime(dfRaw['date']).dt.dayofweek
dfRaw['quarter'] = pd.to_datetime(dfRaw['date']).dt.quarter
dfRaw['dateClean'] = pd.to_datetime(dfRaw['date']).dt.date

In [ ]:
### create model DF
df = dfRaw.copy()
df.dropna(inplace=True)
df.drop_duplicates(inplace=True)
df.drop(columns='date',inplace=True)
df.rename(columns={'dateClean': 'date'}, inplace=True)


,open,high,low,close,volume,adj_high,adj_low,adj_close,adj_open,adj_volume,split_factor,dividend,symbol,exchange,month,year,day,day_of_week,quarter,date
1,604.20,612.14,600.51,608.75,7483600.0,612.1400,600.5100,608.75,604.200,7493479.0,1.0,0.0,VOO,ARCX,10,2025,14,1,4,2025-10-14
2,607.37,611.03,606.58,609.61,6955600.0,611.0272,606.5750,609.61,607.370,6968434.0,1.0,0.0,VOO,ARCX,10,2025,13,0,4,2025-10-13
3,617.99,619.60,600.23,600.51,13600000.0,619.6000,600.2300,600.51,617.990,13664157.0,1.0,0.0,VOO,ARCX,10,2025,10,4,4,2025-10-10
4,619.23,619.59,615.28,617.10,5891900.0,619.5900,615.2800,617.10,619.230,5898496.0,1.0,0.0,VOO,ARCX,10,2025,9,3,4,2025-10-09
5,616.22,618.90,615.46,618.77,5022700.0,618.9000,615.4642,618.77,616.215,5035243.0,1.0,0.0,VOO,ARCX,10,2025,8,2,4,2025-10-08


In [ ]:
### feature engineering for machine learning model
df['per_change'] = ((df['close']-df['open'])/df['open'])*100

df['movement'] = df['per_change'].apply(lambda x: 1 if x > 0.25 else 0)
df.head()

,open,high,low,close,volume,adj_high,adj_low,adj_close,adj_open,adj_volume,...,symbol,exchange,month,year,day,day_of_week,quarter,date,per_change,movement
1,604.20,612.14,600.51,608.75,7483600.0,612.1400,600.5100,608.75,604.200,7493479.0,...,VOO,ARCX,10,2025,14,1,4,2025-10-14,0.753062,1
2,607.37,611.03,606.58,609.61,6955600.0,611.0272,606.5750,609.61,607.370,6968434.0,...,VOO,ARCX,10,2025,13,0,4,2025-10-13,0.368803,1
3,617.99,619.60,600.23,600.51,13600000.0,619.6000,600.2300,600.51,617.990,13664157.0,...,VOO,ARCX,10,2025,10,4,4,2025-10-10,-2.828525,0
4,619.23,619.59,615.28,617.10,5891900.0,619.5900,615.2800,617.10,619.230,5898496.0,...,VOO,ARCX,10,2025,9,3,4,2025-10-09,-0.343976,0
5,616.22,618.90,615.46,618.77,5022700.0,618.9000,615.4642,618.77,616.215,5035243.0,...,VOO,ARCX,10,2025,8,2,4,2025-10-08,0.413813,1


In [82]:
### silver level data final row counts
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 250 entries, 1 to 250
Data columns (total 20 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   open          250 non-null    float64
 1   high          250 non-null    float64
 2   low           250 non-null    float64
 3   close         250 non-null    float64
 4   volume        250 non-null    float64
 5   adj_high      250 non-null    float64
 6   adj_low       250 non-null    float64
 7   adj_close     250 non-null    float64
 8   adj_open      250 non-null    float64
 9   adj_volume    250 non-null    float64
 10  split_factor  250 non-null    float64
 11  dividend      250 non-null    float64
 12  symbol        250 non-null    object 
 13  exchange      250 non-null    object 
 14  month         250 non-null    int32  
 15  year          250 non-null    int32  
 16  day           250 non-null    int32  
 17  day_of_week   250 non-null    int32  
 18  quarter       250 non-null    int32

In [91]:
df.to_csv('silver_stock_info.csv', index=False)

#### Gold Level

In [92]:
### business case is building a model that will predict if a stocks price will move .25% or higher in the day. 
### first step is featuer selection, then scaling/standardization

In [93]:

dfGold = df.copy()

In [94]:
target = dfGold['movement']
features = dfGold[['open', 'high', 'low', 'close', 'volume', 'month', 'year', 'day', 'day_of_week', 'quarter', 'per_change']]